# OncoAssist: Oncology Support Assistant

This assistant provides general information about cancer types and treatment modalities, offers guidance on managing common side effects, suggests support resources, and gives treatment overviews. 

**Disclaimer:** All information provided is for general informational purposes only and is not a substitute for professional medical advice.










### Imports

In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

### Initialization

In [2]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print("OpenAI API Key exists and begins", openai_api_key[:8])
else:
    print("OpenAI API Key not set")

MODEL = "gpt-4o-mini"
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-


### Define Data for Tools

In [3]:
# Data for general cancer information
cancer_info = {
    "breast cancer": "Breast cancer forms in the cells of the breasts. Common treatments include surgery, chemotherapy, radiation, and hormone therapy.",
    "lung cancer": "Lung cancer is often linked to smoking and involves malignant growths in lung tissues. Treatments include surgery, chemotherapy, and targeted therapy.",
    # Add additional cancer types as needed.
}

# Data for side effect management advice
side_effect_advice = {
    "nausea": "For nausea, try eating small, frequent meals and staying well hydrated. If it persists, please consult your healthcare provider.",
    "fatigue": "For fatigue, ensure you rest adequately and maintain light activity as tolerated. Persistent fatigue should be discussed with your doctor.",
    "pain": "For managing pain, over-the-counter pain relievers may help; however, please seek professional guidance for chronic or severe pain.",
}

# Data for support resources
support_resources_data = {
    "national": {
        "American Cancer Society": "https://www.cancer.org",
        "National Cancer Institute": "https://www.cancer.gov",
        "Cancer Support Community": "https://www.cancersupportcommunity.org"
    },
    "local": {
        # Example: region-specific support could be added here.
        "default": "For local support, please consult your healthcare provider or local hospital."
    }
}

# Data for treatment overviews
treatment_overviews = {
    "chemotherapy": "Chemotherapy uses drugs to kill cancer cells. It may be used alone or in combination with other treatments. Side effects vary by patient.",
    "immunotherapy": "Immunotherapy leverages the body's immune system to fight cancer. It is effective for certain cancers with specific markers.",
    "radiation": "Radiation therapy uses high-energy rays to target and kill cancer cells, often shrinking tumors before surgery.",
    "surgery": "Surgery involves the physical removal of tumors and is often the first line of treatment for many cancers.",
}

## Tools

### Tool 1: General Information and Education

In [4]:
def get_general_info(cancer_type):
    info = cancer_info.get(cancer_type.lower(), "No specific information available for this cancer type.")
    disclaimer = "Disclaimer: This information is for educational purposes only and is not a substitute for professional medical advice."
    return f"{info} {disclaimer}"

general_info_function = {
    "name": "get_general_info",
    "description": "Provides general information and education on a specific type of cancer.",
    "parameters": {
        "type": "object",
        "properties": {
            "cancer_type": {
                "type": "string",
                "description": "The type of cancer (e.g., breast cancer, lung cancer)"
            },
        },
        "required": ["cancer_type"],
        "additionalProperties": False
    }
}

### Tool 2: Symptom and Side Effect Management

In [5]:
def get_side_effect_management(side_effect):
    advice = side_effect_advice.get(side_effect.lower(), "No specific advice available for this side effect.")
    disclaimer = "Disclaimer: This advice is general and not a substitute for professional medical care."
    return f"{advice} {disclaimer}"

side_effect_function = {
    "name": "get_side_effect_management",
    "description": "Provides general advice on managing side effects of cancer treatments.",
    "parameters": {
        "type": "object",
        "properties": {
            "side_effect": {
                "type": "string",
                "description": "The side effect you need advice on (e.g., nausea, fatigue, pain)"
            },
        },
        "required": ["side_effect"],
        "additionalProperties": False
    }
}

### Tool 3: Support Resources

In [6]:
def get_support_resources(region=None):
    resources = support_resources_data.get("national")
    # Optionally, include local support if region is specified and available
    if region and region.lower() in support_resources_data.get("local", {}):
        local_support = support_resources_data["local"][region.lower()]
        resources["Local Support"] = local_support
    disclaimer = "Disclaimer: This information is provided for general guidance and should be verified locally."
    return {"resources": resources, "disclaimer": disclaimer}

support_resources_function = {
    "name": "get_support_resources",
    "description": "Provides contact information and links for cancer support resources.",
    "parameters": {
        "type": "object",
        "properties": {
            "region": {
                "type": "string",
                "description": "Optional: Specify a region to get localized support resources."
            },
        },
        "required": [],
        "additionalProperties": False
    }
}

### Tool 4: Treatment Overview

In [7]:
def get_treatment_overview(treatment_type):
    overview = treatment_overviews.get(treatment_type.lower(), "No information available for this treatment type.")
    disclaimer = "Disclaimer: This overview is for informational purposes only. Please consult your oncology team for personalized advice."
    return f"{overview} {disclaimer}"

treatment_overview_function = {
    "name": "get_treatment_overview",
    "description": "Provides an overview of a specific cancer treatment modality.",
    "parameters": {
        "type": "object",
        "properties": {
            "treatment_type": {
                "type": "string",
                "description": "The type of treatment (e.g., chemotherapy, immunotherapy, radiation, surgery)"
            },
        },
        "required": ["treatment_type"],
        "additionalProperties": False
    }
}

### Combining all tools into a list

In [8]:
tools = [
    {"type": "function", "function": general_info_function},
    {"type": "function", "function": side_effect_function},
    {"type": "function", "function": support_resources_function},
    {"type": "function", "function": treatment_overview_function},
]

## Defining the System Message

In [9]:
system_message = """You are OncoAssist, a compassionate oncology support assistant.
You provide general information about cancer types, manage common treatment side effects, suggest support resources, and offer treatment overviews.
Disclaimer: This tool is for general informational purposes only and is not a substitute for professional medical advice. Always consult your healthcare provider for personalized recommendations."""

## Chat Function to Handle Multiple Tool Calls

In [10]:
def chat(message, history):
    # Convert Gradio history format to OpenAI message format
    messages = [{"role": "system", "content": system_message}]
    
    # Add previous conversation from history
    for h in history:
        user_msg, assistant_msg = h
        messages.append({"role": "user", "content": user_msg})
        messages.append({"role": "assistant", "content": assistant_msg})
    
    # Add the current user message
    messages.append({"role": "user", "content": message})
    
    # Call the OpenAI Chat API (or simulate it) with the available tools
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    
    # Handle tool calls (limit to avoid infinite loops)
    max_tool_calls = 3
    tool_call_count = 0
    
    while response.choices[0].finish_reason == "tool_calls" and tool_call_count < max_tool_calls:
        tool_call_count += 1
        bot_message = response.choices[0].message
        messages.append({"role": "assistant", "content": bot_message.content, "tool_calls": bot_message.tool_calls})
        
        # Process each tool call in the response
        for tool_call in bot_message.tool_calls:
            function_name = tool_call.function.name
            arguments = json.loads(tool_call.function.arguments)
            
            # Call the appropriate function based on the function name
            result = call_tool_function(function_name, arguments)
            
            # Add the tool response to the messages
            tool_response = {
                "role": "tool",
                "content": json.dumps(result),
                "tool_call_id": tool_call.id
            }
            messages.append(tool_response)
        
        # Get a new response that includes the tool results
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

### Helper Function to Route the Right Tool Call

In [11]:
def call_tool_function(function_name, arguments):
    if function_name == "get_general_info":
        cancer_type = arguments.get('cancer_type')
        info = get_general_info(cancer_type)
        return {"cancer_type": cancer_type, "info": info}
    
    elif function_name == "get_side_effect_management":
        side_effect = arguments.get('side_effect')
        advice = get_side_effect_management(side_effect)
        return {"side_effect": side_effect, "advice": advice}
    
    elif function_name == "get_support_resources":
        region = arguments.get('region')
        resources = get_support_resources(region)
        return {"region": region, "resources": resources}
    
    elif function_name == "get_treatment_overview":
        treatment_type = arguments.get('treatment_type')
        overview = get_treatment_overview(treatment_type)
        return {"treatment_type": treatment_type, "overview": overview}
    
    return {"error": f"Unknown function: {function_name}"}

### User Interface

In [12]:
# Using Gradio

with gr.Blocks() as demo:
    gr.Markdown("# OncoAI: Oncology Support Assistant")
    
    chatbot = gr.ChatInterface(
        fn=chat,
        chatbot=gr.Chatbot(height=500),
        title="Chat with OncoAI",
        description="Ask about cancer types, treatment options, managing side effects, and support resources.",
        examples=[
            "What is breast cancer?",
            "What can I do for nausea after chemotherapy?",
            "Can you help me find support groups?",
            "Tell me about immunotherapy."
        ]
    )
    
    with gr.Accordion("Available Cancer Types", open=False):
        types = ", ".join([t.title() for t in cancer_info.keys()])
        gr.Markdown(types)
    
    with gr.Accordion("Supported Features", open=False):
        gr.Markdown(
            """
            - General cancer information and education
            - Management of common treatment side effects
            - Guidance on support resources and counseling services
            - Overview of treatment modalities
            """
        )

# Launch the application
demo.launch()

/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/components/chatbot.py:285: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(
/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/chat_interface.py:317: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'tuples', will be used.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7889

To create a public link, set `share=True` in `launch()`.
